In [4]:
!pip install git+https://github.com/v3io/v3io-gputils

  Cloning https://github.com/v3io/v3io-gputils to /tmp/pip-req-build-y88dqrfb
  Running setup.py bdist_wheel for v3io-gputils ... done
  Stored in directory: /tmp/pip-ephem-wheel-cache-14l1hcn_/wheels/31/78/35/87d3ed0349478f78fc174953de2642d7284d30a52e14a243b9
Successfully built v3io-gputils


In [46]:
!git clone http://github.com/tensorflow/benchmarks.git --branch cnn_tf_v1.13_compatible

Cloning into 'benchmarks'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 4174 (delta 21), reused 17 (delta 11), pack-reused 4134
Receiving objects: 100% (4174/4174), 2.17 MiB | 271.00 KiB/s, done.
Resolving deltas: 100% (2793/2793), done.


In [47]:
HOROVOD_JOB_NAME = "horovod-tf-benchmark"

In [58]:
from v3io_gputils.mpijob import MpiJob
import os

job = MpiJob(HOROVOD_JOB_NAME, 'iguaziodocker/horovod:0.1.1', [os.path.join(os.getcwd(), 'benchmarks', 'scripts', 'tf_cnn_benchmarks', 'tf_cnn_benchmarks.py'),
                                                        '--batch_size=32',
                                                        '--model=resnet50',
                                                        '--variable_update=horovod',
                                                        '--use_fp16=true',
                                                        '--xla=true'])
job.replicas(1).gpus(1)
job.submit()


{'apiVersion': 'kubeflow.org/v1alpha1',
 'kind': 'MPIJob',
 'metadata': {'creationTimestamp': '2019-09-18T11:46:13Z',
              'generation': 1,
              'name': 'horovod-tf-benchmark',
              'namespace': 'default-tenant',
              'resourceVersion': '2000563',
              'selfLink': '/apis/kubeflow.org/v1alpha1/namespaces/default-tenant/mpijobs/horovod-tf-benchmark',
              'uid': 'ea7b8b00-da09-11e9-a796-02e0e01c020a'},
 'spec': {'replicas': 1,
          'template': {'spec': {'containers': [{'command': ['mpirun',
                                                            'python',
                                                            '/User/tutorials/demos/horovod/gpu/benchmarks/scripts/tf_cnn_benchmarks/tf_cnn_benchmarks.py',
                                                            '--batch_size=32',
                                                            '--model=resnet50',
                                                            '--

In [61]:
!kubectl get pods | grep $HOROVOD_JOB_NAME

horovod-tf-benchmark-launcher-rnvts                 1/1     Running            0          59s
horovod-tf-benchmark-worker-0                       1/1     Running            0          62s


In [62]:
!kubectl get mpijob $HOROVOD_JOB_NAME -o yaml

apiVersion: kubeflow.org/v1alpha1
kind: MPIJob
metadata:
  creationTimestamp: 2019-09-18T11:46:13Z
  generation: 4
  name: horovod-tf-benchmark
  namespace: default-tenant
  resourceVersion: "2000607"
  selfLink: /apis/kubeflow.org/v1alpha1/namespaces/default-tenant/mpijobs/horovod-tf-benchmark
  uid: ea7b8b00-da09-11e9-a796-02e0e01c020a
spec:
  backoffLimit: 6
  replicas: 1
  template:
    metadata:
      creationTimestamp: null
    spec:
      containers:
      - command:
        - mpirun
        - python
        - /User/tutorials/demos/horovod/gpu/benchmarks/scripts/tf_cnn_benchmarks/tf_cnn_benchmarks.py
        - --batch_size=32
        - --model=resnet50
        - --variable_update=horovod
        - --use_fp16=true
        - --xla=true
        image: iguaziodocker/horovod:0.1.1
        name: horovod-tf-benchmark
        resources:
          limits:
            nvidia.com/gpu: "1"
        securityContext:
          capabilities:
            add:
            - IPC_LOCK
        volum

In [63]:
job.delete()

{'apiVersion': 'v1',
 'details': {'group': 'kubeflow.org',
             'kind': 'mpijobs',
             'name': 'horovod-tf-benchmark',
             'uid': 'ea7b8b00-da09-11e9-a796-02e0e01c020a'},
 'kind': 'Status',
 'metadata': {},
 'status': 'Success'}
